# Trabalho Prático 3 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Esquemas pós-quânticos de assinaturas digitais* - CRYSTALS-Dilithium

Nota: Baseado no documento CRYSTALS-Dilithium (https://pq-crystals.org/dilithium/data/dilithium-specification-round3-20210208.pdf)

### Detalhes acerca do algorítmo base

Dos três esquemas pós-quânticos de assinaturas digitais propostos no enunciado do trabalho (*qTesla*, *Dilithium* e *Rainbow*), um dos que escolhemos fazer foi o **Dilithium** cuja segurança se baseia na dificuldade em encontrar vetores curtos em *lattices*. Para além disto e indo ao encontro do que é mencionado do documento mencionado acima, o 
**Dilithium** foi desenhado tendo em mente os seguintes critérios:

  - Simple to implement securely
  - Be conservative with parameters
  - Minimize the size of public key + signature
  - Be modular – easy to vary security

De seguida, tentaremos explicar unicamente as três principais funções que devem ser chamadas nesta classe: **geraChaves()**, **assina()** e **verifica()**.

**Geração da chave pública *($\rho$,$t_1$)* e da chave privada ($\rho$,K,tr,$s_1$,$s_2$,$t_0$) *(função geraChaves())***

O algorítmo de geração de chaves gera uma **matriz A**, k $\times$ l, para a qual cada uma das suas entradas é um **polinímio** pertencente ao **anel** **Rq = Zq[X]/(Xn + 1)**. Importa referir que, para este contexto em particular, , temos sempre que **q = $2^{23}$ − $2^{13}$ + 1** e **n = 256**. De seguida, o algorítmo trata de gerar 2 vetores privados, **s1** e **s2**. Finalmente, a segunda parte da chave pública é calculada como sendo **t = As1 + s2**. Todas as operações algébricas neste esquema são assumidas como sendo sobre o *polynomial ring* **Rq**.


**Assinatura *(função assina())***

O algorítmo de **assinatura** gera um *masking vector* de polinómios, y com coeficientes menores que $\gamma_1$. Esse mesmo parâmetro é definido de forma estratégica – **é grande o suficiente para que a assinatura não revele a chve secreta** (i.e. o algorítmo de assinatura é
*zero-knowledge*), mas também **pequeno o suficiente para que a assinatura não seja facilmente forjada**.

De seguida, é computado **Ay**
then computes Ay and sets w1 to be the “high-order” bits of the coefficients in this
vector
  

**Verificação *(função verifica())***

  - Recebe como argumentos a mensagem original, a assinatura digital a verificar e a chave pública;
  - Inicialmente, começa por calcular o encoding **c** através da componente **c'** da assinatura digital;
  - Depois, calcula-se **w** através da expressão: *w = a*z - t*c* pertencente a **Rq**, sendo que este resultado também deve ser arredondado para ficar entre [-q/2,q/2-1];
  - A seguir, verifica-se se **c'** é diferente do hash de **v** com o hash da mensagem, pelo que se for retorna -1 e a verificação falha, caso contrário retorna 0 e a assinatura é válida.
  


In [4]:
import hashlib

class Dilithium(object):
    
    def __init__(self, q, d, tau, chall_ent, gama1, gama2, k, l, eta, beta, omega, reps, timeout=None):
        
        self.q = q
        self.d = d
        self.tau = tau
        self.chall_ent = chall_ent
        self.gama1 = gama1
        self.gama2 = gama2
        self.k = k
        self.l = l
        self.eta = eta
        self.beta = beta
        self.omega = omega
        self.reps = reps
          

    '''
    Implementação da função H() responsável por gerar 
    um digest com 256-bit de comprimento (32 bytes)
    '''        
    def H(self, x):
        
        h = hashlib.shake_256()
        h.update(x)
        return h.digest(int(32))
    
    '''
    Implementação da função H() responsável por gerar 
    um digest com (256+512+256)-bit (128 bytes) de comprimento
    '''
    def H3(self, x):
        
        h = hashlib.shake_256()
        h.update(x)
        buffer = h.digest(int(128))
        return (buffer[:32], buffer[32:96], buffer[-32:])
    
    '''
    Implementação da função SampleInBall descrita 
    no documento onde se encontra a especificação 
    do dilithium
    '''
    def SampleInBall(self, rho):
        
        # Initialize c = c0c1 . . . c255 = 00 . . . 0
        random.seed(a=rho, version=2)
        c = []
        for i in range(256):
            c.append(0)
        # for i := 256 − tau to 255
        for i in range(256 - self.tau, 256):
            # j <- {0, 1, . . . , i}
            j = random.randint(0, i)
            # s <- {0, 1}
            s = random.randint(0, 1)
            # ci := cj
            c[i] = c[j]
            # cj := (−1)^s
            c[j] = (-1)^s
        
        return c
    
    '''
    Implementação da função Power2Round
    '''
    def Power2Round(self, r, d):
        
        # r := r mod+ q
        r = r % self.q
        # r := mod ± 2^d
        r0 = r % (2^d)
        r0 = r0 - (2^(d-1))
        # return ((r − r0)/2^d, r0)
        return ((r - r0)//(2^d)), r0
    
    '''
    Implementação da função MakeHint
    '''
    def MakeHint(self, z, r, alpha):
        
        # r1 := HighBits_q(r, alpha)
        r1 = self.HighBits(r, alpha)
        # v1 := HighBits_q(r + z, alpha)
        v1 = self.HighBits(r + z, alpha)
        
        # return [r1 != v1]
        return r1 != v1
    
    
    '''
    Implementação da função UseHint
    '''
    def UseHint(self, h, r, alpha):
        
        # m := (q − 1)/alpha
        m = (self.q - 1)//alpha
        # (r1, r0) := Decompose_q(r, alpha)
        (r1, r0) = self.Decompose(r, alpha)
        # if h = 1 and r0 > 0 return (r1 + 1) mod+ m
        if h == 1 & r0 > 0:
            return (r + 1) % m
        # if h = 1 and r0 <= 0 return (r1 − 1) mod+ m
        if h == 1 & r0 <= 0:
            return (r - 1) % m
        return r1
    
    '''
    Implementação da função HighBits
    '''
    def HighBits(self, r, alpha):
        
        # (r1, r0) := Decomposeq(r, alpha)
        (r1, r0) = self.Decompose(r, alpha)
        return r1
    
    '''
    Implementação da função LowBits
    '''
    def LowBits(self, r, alpha):
        
        # (r1, r0) := Decomposeq(r, alpha)
        (r1, r0) = self.Decompose(r, alpha)
        return r0
    
    '''
    Implementação da função Decompose
    '''
    def Decompose(self, r, alpha):
        
        # r := r mod+ q
        r = r % self.q
        # r0 := r mod± alpha
        r0 = r % alpha
        r0 = r0 - (alpha//2)
        # if r − r0 = q − 1 then r1 := 0; r0 := r0 − 1
        if r - r0 == self.q - 1:
            r1 = 0
            r0 = r0 - 1
        else:
            # else r1 := (r − r0)/alpha
            r1 = (r - r0)//alpha
        
        return (r1, r0)
    
    '''
    Função que representa o seguinte comportamento: 
    modulo_infinito(w) < condição
    '''
    def size_of_elements_low(self, w, cond):
        
        x = w % self.q
        x = x - (self.q//2)
        if x < 0:
            x = -x
        
        return x < cond
    
    '''
    Função que representa o seguinte comportamento: 
    modulo_infinito(w) >= condição
    '''
    def size_of_elements_bige(self, w, cond):
        
        x = w % self.q
        x = x - (self.q//2)
        if x < 0:
            x = -x
        
        return x >= cond
    
    '''
    Função que representa a seguinte condição:
    # 1's in w is greater than $cond
    '''
    def number_of_1s_big(self, w, cond):
        
        counter = 0
        aux = w
        while aux > 0:
            if aux % 2:
                counter += 1
            aux = aux // 2
        
        return counter > cond
    
    '''
    Função que representa a seguinte condição:
    # 1's in w is lower or equal than $cond
    '''
    def number_of_1s_lowe(self, w, cond):
        
        counter = 0
        aux = w
        while aux > 0:
            if aux % 2:
                counter += 1
            aux = aux // 2
        
        return counter <= cond
    
    '''
    Implementação da função ExpandA
    '''
    def ExpandA(self, x):
        
        return 0
    
    '''
    Implementação da função ExpandS
    '''
    def ExpandS(self, x):
        
        return 0
    
    '''
    Implementação da função ExpandMask
    '''
    def ExpandMask(self, x1, x2):
        
        return 0
    
    '''
    Implementação do algorítmo NTT a um 
    vetor cujos elementos pertencem a Rq
    '''
    def NTTvec(self, a):
        
        psi = []
        t = 2^13
        m = 1
        while m < 2^13:
            t = t//2
            for i in range(m):
                j1 = 2 * i * t
                j2 = j1 + t - 1
                S = 'sadsadasdsadasda'
                for j in range(j1, j2+1):
                    U = a[j]
                    V = a[j + t] * S
                    a[j] = (U + V) % q
                    a[j + t] = (U - V) % q
            m = m*2
        return a
    
    '''
    Implementação do algorítmo de inversa de NTT a um 
    vetor cujos elementos pertencem a Rq
    '''
    def INTTvec(self, a):
        
        return 0
    
    '''
    Implementação do algorítmo de NTT a uma
    matrix cujos elementos pertencem a Rq
    '''
    def NTTmat(self, M):
        
        return 0
    
    '''
    Implementação do algorítmo de inversa de NTT a uma
    matrix cujos elementos pertencem a Rq
    '''
    def INTTmat(self, M):
        
        return 0
    
    '''
    Função responsável pela geração de ambas 
    as chaves, pública e privada.
    '''
    def geraChaves(self):
        
        # zeta <- {0, 1}^256
        zeta = os.urandom(32)
        # (rho, rho',K) € {0, 1}^256 × {0, 1}^512 × {0, 1}^256 := H(zeta)
        (rho, rho0, K) = self.H3(zeta)
        # A € Rq^k×q := ExpandA(rho)
        A = self.ExpandA(rho)
        Acirc = self.NTTmat(A)
        #(s1, s2) € Sn^l × Sn^k := ExpandS(rho')
        (s1, s2) = self.ExpandS(rho0)
        # Compute As1 as NTT^−1(ˆA · NTT(s1))
        As1 = self.INTTvec(Acirc*self.NTTvec(s1))
        # t := As1 + s2
        t = As1 + s2
        # (t1, t0) := Power2Roundq(t, d)
        (t1, t0) = self.Power2Round(t,self.d)
        # tr € {0, 1}^256 := H(rho || t1)
        tr = self.H(rho + t1)
     
        pk = (rho, t1)
        sk = (rho, K, tr, s1, s2, t0)
        return pk, sk
    
    '''
    Função responsável por assinar uma mensagem 
    'M' com uma determinada chave privada 'sk'
    '''
    def assina(self, sk, M):
        
        (rho, K, tr, s1, s2, t0) = sk
        
        # A € Rq^k×q := ExpandA(rho)
        A = self.ExpandA(rho)
        Acirc = self.NTTmat(A)
        # μ € {0, 1}^512 := H(tr || M)
        niu = self.H(tr + M)
        # k := 0, (z, h) := err0
        kappa = 0
        z = 0; h = 0
        # rho' € {0, 1}^512 := H(K || μ) (or rho' <- {0, 1}^512 for randomized signing)
        rhol = self.H(K + niu)
        # ˆs1 := NTT(s1)
        s1circ = self.NTTvec(s1)
        # ˆs2 := NTT(s2)
        s2circ = self.NTTvec(s2)
        # ˆt0 := NTT(t0)
        t0circ = self.NTTvec(t0)
        # while (z, h) = erro
        while z == 0 & h == 0:
            
            # y € ˜S_gama1^l := ExpandMask(rho', k)
            y = self.ExpandMask(rhol, kappa)
            # w := Ay, ou w := NTT−1(ˆA · NTT(y))
            w = self.INTTvec(Acirc * NTTvec(y))
            # w1 := HighBitsq(w, 2*gama2)
            w1 = self.HighBits(w, 2 * self.gama2)
            # ˜c € {0, 1}^256 := H(μ || w1)
            ctil = self.H(niu, w1)
            # c € B_tau := SampleInBall(˜c)
            c = self.SampleInBall(ctil)
            # Store c in NTT representation as ˆc = NTT(c)
            ccirc = self.NTT(c)
            # Compute cs1 as NTT−1(ˆc · ˆs1)
            cs1 = self.INTTvec(ccirc*s1circ)
            # z := y + cs1
            z = y + cs1
            # Compute cs2 as NTT−1(ˆc · ˆs2)
            cs2 = self.INTTvec(ccirc*s2circ)
            # r0 := LowBitsq(w − cs2, 2*gama2)
            r0 = self.LowBits(w-cs2, 2 * self.gama2)
            # if ||z||inf >= gama1 − beta or ||r0||inf >= gama2 − beta, then (z, h) := erro
            if self.size_of_elements_bige(z, self.gama1 - self.beta) | self.size_of_elements_bige(e0, self.gama2 - self.beta):
                z = 0
                h = 0
            else:
                # Compute ct0 as NTT−1(ˆc ·ˆt0)
                ct0 = self.INTTvec(ccirc * t0circ)
                # h := MakeHintq(−ct0,w − cs2 + ct0, 2*gama2)
                h = self.MakeHint(-ct0, w - cs2 + ct0, 2 * self.gama2)
                # if ||ct0||inf >= 2 or the # of 1’s in h is greater than omega, then (z, h) := erro
                if self.size_of_elements_bige(ct0 , self.gama2) | self.number_of_1s_big(h, self.omega):
                    z = 0
                    h = 0
            # k := k + l
            kappa = kappa + l
    
        sigma = (ctil, z, h)
        return sigma
    
    '''
    Função que verifica a assinatura de uma mensagem 
    com uma determinada chave pública
    '''
    def verifica(self, pk, M, sigma):
        
        (rho, t1) = pk
        (ctil, z, h) = sigma
        # A € Rq^k×q := ExpandA(rho)
        A = self.ExpandA(rho)
        Acirc = self.NTTmat(A)
        # μ € {0, 1}^512 := H(H(rho || t1) || M)
        niu = self.H(self.H(rho + t1) + M)
        # c := SampleInBall(˜c)
        c = self.SampleInBall(ctil)
        # w0 1 := UseHintq(h, Az − ct1 · 2^d, 2*gama2) -> Compute as NTT−1(ˆA · NTT(z)−NTT(c) · NTT(t1 · 2^d))
        wl1 = self.UseHint(h, self.INTTvec((Acirc * self.NTTvec(z)) - (self.NTTvec(c) * self.NTTvec(t1*(2^13)))))
        # return [||z||inf < gama1 − beta] and [˜c = H(μ || w'1 )] and [# of 1’s in h is <= omega]
        return self.size_of_elements_low(z, self.gama1 - self.beta) & ctil == self.H(niu + wl1) & self.number_of_1s_lowe(h, self.omega)
    
    
    

#### Testagem da classe definida acima:

In [5]:
# Parametros iniciais
q = 8380417
# 
d = 13
# 
tau = 39
# 
chall_ent = 192
# 
gama1 = 2^17
# 
gama2 = 95232
# 
k = 4
# 
l = 4
# 
eta = 2
#
beta = 78
#
omega = 80
#
reps = 4.25

# Teste
print("[Testagem da classe acima]:\n")

dilithium = Dilithium(q, d, tau, chall_ent, gama1, gama2, k, l, eta, beta, omega, reps)

pub,priv = dilithium.geraChaves()

[Testagem da classe acima]:



TypeError: cannot unpack non-iterable sage.rings.integer.Integer object